In [1]:
import os
import shutil
import json
import time

import requests

import random

import av
import cv2
import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import albumentations as A

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoProcessor, AutoModel, pipeline

2024-06-11 10:24:08.143381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 10:24:08.711011: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/cv2/../../lib64:
2024-06-11 10:24:08.711060: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vladimir/.virtualenvs/ml/lib/python3.10/site-pa

In [2]:
data = json.load(open('videollava.json'))

In [3]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [4]:
batch_size = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
random.shuffle(data)

In [6]:
X_train, X_val = data[:-100], data[-100:]

In [7]:
def apply_video_augmentations(video, transform):
    targets={'image': video[0]}
    for i in range(1, video.shape[0]):
        targets[f'image{i}'] = video[i]
    transformed = transform(**targets)
    transformed = np.concatenate(
        [np.expand_dims(transformed['image'], axis=0)] 
        + [np.expand_dims(transformed[f'image{i}'], axis=0) for i in range(1, video.shape[0])]
    )
    return transformed

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    while converted_len >= seg_len and clip_len > 1:
        clip_len -= 1
        converted_len = int(clip_len * frame_sample_rate)
    end_idx = converted_len
    start_idx = 0
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [8]:
from FlagEmbedding import BGEM3FlagModel

bge = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cpu') # Setting use_fp16 to True speeds up computation with a slight performance degradation

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [9]:
processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch16")
model = AutoModel.from_pretrained("microsoft/xclip-base-patch16")
model.to(device)

projector = nn.Linear(512, 1024, bias=False)
projector.to(device)

Linear(in_features=512, out_features=1024, bias=False)

In [10]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.5)
], additional_targets={
    f'image{i}': 'image'
    for i in range(1, 8)
})

In [11]:
def download_video(video_url):
    try:
        filename = '_'.join(video_url.split('/')[4:])
        if os.path.exists(f'videos/{filename}'):
            return filename
        
        response = requests.get(video_url, timeout=300)
        
        with open(f'videos/{filename}', 'wb') as file:
            file.write(response.content)
        return filename
    except Exception:
        return ''

In [12]:
class VideoDataset(Dataset):

    def __init__(self, meta, transform=None):
        self.meta = meta
        self.transform = transform

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

            
        while True:
            try:
                
                file_path = download_video(self.meta[idx]['link'])
                container = av.open(f'videos/{file_path}')
                frame_sample_rate = random.randint(5, 50)
                indices = sample_frame_indices(clip_len=8, frame_sample_rate=frame_sample_rate, seg_len=container.streams.video[0].frames)
                video = read_video_pyav(container, indices)
                while video.shape[0] < 8:
                    video = np.vstack([video, video[-1:]])
            except Exception as e:
                print(e)
                idx = random.randint(0, len(self.meta))
                continue
                
            break

        if self.transform:
            video = apply_video_augmentations(video, self.transform)

        inputs = processor(
            text='',
            videos=list(video),
            return_tensors="pt",
        )
        
        for i in inputs:
            inputs[i] = inputs[i][0]

        return inputs, self.meta[idx]['description']

In [13]:
train_dataset = VideoDataset(meta=X_train, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

val_dataset = VideoDataset(meta=X_val, transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [14]:
epochs = 5
lr = 1e-5
projector_lr = 1e-3

optimizer = optim.AdamW(model.parameters(), lr)
projector_optimizer = optim.AdamW(projector.parameters(), projector_lr)


In [15]:
for param in model.parameters():
    param.requires_grad = False

In [17]:
for epoch in range(epochs):

    model.eval() 
    projector.train() 

    train_loss = []
    for i, (batch, texts) in enumerate(tqdm(train_dataloader, desc=f"Epoch: {epoch}")):

        text_embeddings = bge.encode(
            texts, 
            batch_size=1, 
            max_length=1024, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
        )['dense_vecs']
        text_embeddings = torch.tensor(text_embeddings).to(device)
        
        optimizer.zero_grad()
        projector_optimizer.zero_grad()

        batch = batch.to(device)

        outputs = projector(model(**batch).video_embeds)

        loss = (1 - nn.functional.cosine_similarity(outputs, text_embeddings)).mean()
        loss.backward()
        optimizer.step()
        projector_optimizer.step()

        train_loss.append(loss.item())
        

    print('Training loss:', np.mean(train_loss))
    
    model.eval()
    projector.eval() 

    val_loss = []
    for i, (batch, texts) in enumerate(tqdm(val_dataloader, desc=f"Epoch: {epoch}")):

        text_embeddings = bge.encode(
            texts, 
            batch_size=1, 
            max_length=1024, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
        )['dense_vecs']
        text_embeddings = torch.tensor(text_embeddings).to(device)

        batch = batch.to(device)

        with torch.no_grad():
            outputs = projector(model(**batch).video_embeds)

            loss = (1 - nn.functional.cosine_similarity(outputs, text_embeddings)).mean()
    
            val_loss.append(loss.item())

    print('Val loss:', np.mean(val_loss))
    torch.save(projector.state_dict(), 'projector.pth')

Epoch: 0:   0%|                                                                                                                                                                                                    | 0/1237 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Training loss: 0.18587193483292527


Epoch: 0:   0%|                                                                                                                                                                                                      | 0/13 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Val loss: 0.17087138157624465


Epoch: 1:   0%|                                                                                                                                                                                                    | 0/1237 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

...

Val loss: 0.16633060001409972


Epoch: 4:   0%|                                                                                                                                                                                                    | 0/1237 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Training loss: 0.16013166070490376


Epoch: 4:   0%|                                                                                                                                                                                                      | 0/13 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Val loss: 0.16649101903805366


In [18]:
for param in model.parameters():
    param.requires_grad = True
for param in model.text_model.parameters():
    param.requires_grad = False

In [19]:
epochs = 15

In [ ]:
for epoch in range(epochs):

    model.train() 
    projector.train() 

    train_loss = []
    for i, (batch, texts) in enumerate(tqdm(train_dataloader, desc=f"Epoch: {epoch}")):

        text_embeddings = bge.encode(
            texts, 
            batch_size=1, 
            max_length=1024, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
        )['dense_vecs']
        text_embeddings = torch.tensor(text_embeddings).to(device)
        
        optimizer.zero_grad()
        projector_optimizer.zero_grad()

        batch = batch.to(device)

        outputs = projector(model(**batch).video_embeds)

        loss = (1 - nn.functional.cosine_similarity(outputs, text_embeddings)).mean()
        loss.backward()
        optimizer.step()
        projector_optimizer.step()

        train_loss.append(loss.item())
        

    print('Training loss:', np.mean(train_loss))
    
    model.eval()
    projector.eval() 

    val_loss = []
    for i, (batch, texts) in enumerate(tqdm(val_dataloader, desc=f"Epoch: {epoch}")):

        text_embeddings = bge.encode(
            texts, 
            batch_size=1, 
            max_length=1024, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
        )['dense_vecs']
        text_embeddings = torch.tensor(text_embeddings).to(device)

        batch = batch.to(device)

        with torch.no_grad():
            outputs = projector(model(**batch).video_embeds)

            loss = (1 - nn.functional.cosine_similarity(outputs, text_embeddings)).mean()
    
            val_loss.append(loss.item())

    print('Val loss:', np.mean(val_loss))

Epoch: 0:   0%|                                                                                                                                                                                                    | 0/1237 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Training loss: 0.15413371657857802


Epoch: 0:   0%|                                                                                                                                                                                                      | 0/13 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Val loss: 0.15706428656211266


Epoch: 1:   0%|                                                                                                                                                                                                    | 0/1237 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

...

Epoch: 9:   0%|                                                                                                                                                                                                    | 0/1237 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Training loss: 0.09490222948254745


Epoch: 9:   0%|                                                                                                                                                                                                      | 0/13 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

Val loss: 0.15545902000023767


Epoch: 10:   0%|                                                                                                                                                                                                   | 0/1237 [00:00<?, ?it/s]/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/home/vladimir/.virtualenvs/ml/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  retur

In [ ]:
model.save_pretrained('finetuned-xclip-base-patch16')
processor.save_pretrained('finetuned-xclip-base-patch16')
torch.save(projector.state_dict(), 'projector.pth')